Task objective:

- Define an ES analyzer for Polish texts containing:
    - standard tokenizer
    - synonym filter with alternative forms for months, e.g. wrzesień, wrz, IX.
    - lowercase filter
    - Morfologik-based lemmatizer
    - lowercase filter (looks strange, but Morfologi produces capitalized base forms for proper names, so we have to lowercase them once more).
- Define another analyzer for Polish, without the synonym filter.
- Define an ES index for storing the contents of the corpus from lab 1 using both analyzers. Use different names for the fields analyzed with a different pipeline.
- Load the data to the ES index.
- Determine the number of documents containing the word styczeń (in any form) including and excluding the synonyms.
- Download the QA pairs for the FIQA dataset.
- Compute NDCG@5 for the QA dataset (the test subset) for the following setusp:
    - synonyms enabled and disabled,
    - lemmatization in the query enabled and disabled.

In [2]:
import elasticsearch
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
from datasets import load_dataset
from elasticsearch_dsl import Search, Document, Text

/Users/milenabiernacka/anaconda3/envs/PJN/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
es = Elasticsearch(['http://localhost:9200'])
es.ping()

True

## Indexes

### Analyzer with synonyms

In [4]:
index_with_synonyms_settings = {
    "analysis": {
        "analyzer": {
            "polish_with_synonyms_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": [
                    "lowercase",
                    "polish_months_synonyms",
                    "morfologik_stem",
                    "lowercase"
                ]
            }
        },
        "filter": {
            "polish_months_synonyms": {
                "type": "synonym",
                "synonyms": [
                    "styczeń, sty, I",
                    "luty, lut, II",
                    "marzec, mar, III",
                    "kwiecień, kwi, IV",
                    "maj, V",
                    "czerwiec, cze, VI",
                    "lipiec, lip, VII",
                    "sierpień, sie, VIII",
                    "wrzesień, wrz, IX",
                    "październik, paź, X",
                    "listopad, lis, XI",
                    "grudzień, gru, XII"
                ]
            }
        }
    }
}
index_with_synonyms_mappings = {
    "properties": {
        "text": {
            "type": "text",
            "analyzer": "polish_with_synonyms_analyzer"
        }
    }
}

In [6]:

# tworzenie indeksu
es.indices.create(index="index_with_synonyms", 
                  settings=index_with_synonyms_settings,
                  mappings=index_with_synonyms_mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_with_synonyms'})

### Analyzer without synonyms

In [5]:
index_without_synonyms_settings = {
    "analysis": {
        "analyzer": {
            "polish_without_synonyms_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": [
                    "lowercase",
                    "morfologik_stem",
                    "lowercase"
                ]
            }
        }
    }
}
index_without_synonyms_mappings = {
    "properties": {
        "text": {
            "type": "text",
            "analyzer": "polish_without_synonyms_analyzer"
        }
    }
}

In [39]:
# tworzenie indeksu
es.indices.create(index="index_without_synonyms", 
                  settings=index_without_synonyms_settings,
                  mappings=index_without_synonyms_mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_without_synonyms'})

## Loading dataset FIQA-PL

In [6]:
dataset = load_dataset("clarin-knext/fiqa-pl", "corpus")
df = pd.DataFrame(dataset['corpus'])
df_text = df['text']

In [7]:
df_text

0        Nie mówię, że nie podoba mi się też pomysł szk...
1        Tak więc nic nie zapobiega fałszywym ocenom po...
2        Nigdy nie możesz korzystać z FSA dla indywidua...
3        Samsung stworzył LCD i inne technologie płaski...
4        Oto wymagania SEC: Federalne przepisy dotycząc...
                               ...                        
57633    >Cóż, po pierwsze, drogi to coś więcej niż hob...
57634    Tak, robią. Na dotacje dla firm farmaceutyczny...
57635    >To bardzo smutne, że nie rozumiesz ludzkiej n...
57636    „Czy Twój CTO pozwolił dużej grupie użyć „„adm...
57637    Zapewnienie rządowi większej kontroli nad dyst...
Name: text, Length: 57638, dtype: object

## Loading data to ES

In [42]:
# Iteruje po tekstach i indeksuje je do obydwu stworzonych indeksow: z synonimami i bez
for idx, text in enumerate(df_text):
    document = {
        "text": text,
    }
    es.index(index="index_with_synonyms", id=idx, document=document)
    es.index(index="index_without_synonyms", id=idx, document=document)

In [8]:
# sprawdzenie czy liczba dokumentow zapisanych w ES i index_with_synonyms rowna sie ilosci tekstow z datasetu
if es.count(index="index_with_synonyms")['count'] == len(dataset['corpus']['text']):
    print("Data successfully uploaded.")

Data successfully uploaded.


In [10]:
# sprawdzenie czy liczba dokumentow zapisanych w ES i index_without_synonyms rowna sie ilosci tekstow z datasetu
if es.count(index="index_without_synonyms")['count'] == len(dataset['corpus']['text']):
    print("Data successfully uploaded.")

Data successfully uploaded.


In [9]:
# sprawdzanie liczby zaindeksowanych tesktow
print(es.count(index="index_with_synonyms")['count'])
print(es.count(index="index_without_synonyms")['count'])

57638
57638


## Searching number of documents containing `styczeń` word with and without synonyms

In [11]:
# Zapytanie z synonimami
query_styczen_with_synonyms = {
    "query": {
        "match": {
            "text": {
                "query": "styczeń",
                "analyzer": "polish_with_synonyms_analyzer"
            }
        }
    }
}

# Wykonanie zapytania
response = es.search(index="index_with_synonyms", body=query_styczen_with_synonyms)

# Liczba pasujących dokumentów
total_hits = response["hits"]["total"]["value"]
print("Number of documents including 'styczen' with synonyms: " + str(total_hits))

Number of documents including 'styczen' with synonyms: 10000


In [12]:
# Zapytanie bez synonimow
query_styczen_without_synonyms = {
    "query": {
        "match": {
            "text": {
                "query": "styczeń",
                "analyzer": "polish_without_synonyms_analyzer"
            }
        }
    }
}

# Wykonanie zapytania
response = es.search(index="index_without_synonyms", body=query_styczen_without_synonyms)

# Liczba pasujących dokumentów
total_hits = response["hits"]["total"]["value"]
print("Number of documents including 'styczen' without synonyms: " + str(total_hits))

Number of documents including 'styczen' without synonyms: 329


In [ ]:
# Usunięcie wszystkich dokumentów w danym indeksie -  na wypadek gdybym znowu uruchomiła wczytywanie dokumnetow :/ 
# es.delete_by_query(index="polish_analyzer", body={"query": {"match_all": {}}})

## Loading dataset FIQA-PL-QRELS

In [13]:
dataset_QA = load_dataset("clarin-knext/fiqa-pl-qrels")
dataset_QA_test = dataset_QA['test']
df_qa_test = pd.DataFrame(dataset_QA['test'])

dataset_queries = load_dataset("clarin-knext/fiqa-pl", "queries")
df_queries = pd.DataFrame(dataset_queries['queries'])

In [14]:
df_queries

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...
...,...,...,...
6643,4102,,"Jak mogę ustalić, czy moja stopa zwrotu jest „..."
6644,3566,,"Gdzie mogę kupić akcje, jeśli chcę zainwestowa..."
6645,94,,Wykorzystywanie punktów kart kredytowych do op...
6646,2551,,Jak znaleźć tańszą alternatywę dla tradycyjnej...


In [15]:
# funkcja obliczajaca metryke NDCG przyjmujaca wynik jako parametr
K = 5

def calc_ndcg_k(scores):
    if len(scores) != K : Exception("Invalid scores arr size, != 5")
    dcg = np.sum(scores / np.log2(np.arange(2, len(scores) + 2)))
    idcg = np.sum(sorted(scores, reverse=True) / np.log2(np.arange(2, len(scores) + 2)))
    ndcg = dcg / idcg if idcg > 0 else 0.0
    return ndcg

df_queries_text = df_queries['text']
arr = np.array([0.0 for i in range(K)])
tmp = set()



## 3 versions of NDCG@5

In [16]:
# zapytanie z synonimami
query_ndcg_with_synonyms = {
        "match": {
            "text":{
                "query":"",
                "analyzer":"polish_with_synonyms_analyzer"
            }
        }
    }

# zapytanie bez lematyzacji
query_ndcg_without_lemmatizaion = {
        "match": {
            "text":{
                "query":"",
                "analyzer":"standard"
            }
        }
    }

# zapytanie bez synonimow
query_ndcg_without_synonyms = {
        "match": {
            "text":{
                "query":"",
                "analyzer":"polish_without_synonyms_analyzer"
            }
        }
    }

In [17]:
# funkcja obliczajaca ndcg dla podanego indexu i zapytania
def ndcg_for_index(index, query):
    ndcg = 0
    iterator = 0

    for query_id in df_qa_test['query-id'].unique(): # iteruje po unikalnych query_id
        query_up = df_queries[df_queries['_id'] == str(query_id)].iloc[0]['text'] # pobieram query (jako tekst) do pasujacego query_id
        query['match']['text']['query'] = query_up # update'uje zapytanie do obliczania ndcg o znalezione query
        resp = es.search(index=index, query=query) # znajduje wyniki zapytania
        corpus_ids = df_qa_test[df_qa_test['query-id'] == query_id]['corpus-id'] # znajduje wszystkie corpus_id dla odpowiadajacego query_id
        
        # przechowuje w tmp indeksy, ktore pasuja do zapytania
        tmp = set() 
        for idx in corpus_ids:
            _id = df[df['_id'] == str(idx)].index.tolist()[0]
            tmp.add(_id)
            
        # przegladam K wynikow z ES i sprawdzam czy sa one w tmp. Jeśli są -> ocena 3, jeśli nie -> 0. 
        for idx, val in enumerate(resp['hits']['hits'][:K]):
            _id = np.float64(val['_id'])
            if _id in tmp:
                arr[idx] = 3
            else:
                arr[idx] = 0

        # obliczam NDCG
        ndcg += calc_ndcg_k(arr)
        iterator += 1
        mean_ndcg = ndcg / iterator
    return ("NDCG: "+ str(ndcg), "Mean NDCG: " + str(mean_ndcg))

In [18]:
ndcg_for_index("index_with_synonyms", query_ndcg_with_synonyms)

('NDCG: 172.96008915896866', 'Mean NDCG: 0.2669137178379146')

In [19]:
ndcg_for_index("index_without_synonyms", query_ndcg_without_synonyms)

('NDCG: 172.19452861340906', 'Mean NDCG: 0.2657322972429152')

In [7]:
# Tworze indeks do wersji z lematyzacja
index_without_lemmatization_mappings = {
    "properties": {
        "text": {
            "type": "text",
            "analyzer": "standard"
        }
    }
}

In [8]:
es.indices.create(index = "index_without_lemmatization", mappings=index_without_lemmatization_mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_without_lemmatization'})

In [22]:

ndcg_for_index("index_without_lemmatization", query_ndcg_without_lemmatizaion)

('NDCG: 324.0', 'Mean NDCG: 0.5')